In [4]:
from sklearn.ensemble import RandomForestClassifier
from skopt.space import Integer
from skopt.utils import use_named_args
from skimage.feature import hog
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import csv
from skopt import gp_minimize
import cv2
import concurrent.futures
from os import listdir
from os.path import isfile, join
import numpy as np

path = '../data'
categorias = ['NORMAL', 'PNEUMONIA']

# Definir el espacio de búsqueda para los hiperparámetros del HOG
espacio_hog = [
    Integer(4, 9, name='orientations'),
    Integer(6, 16, name='pixels_per_cell'),
]

def procesar_imagen(imagen_path, hog_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    fd = hog(imagen, orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'],
             cells_per_block=(1,1), visualize=False)

    return fd, etiqueta

def cargar_datos(ruta, hog_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, hog_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

# Nombre del archivo CSV para los resultados
nombre_archivo_csv = "../results/hog_lbp_parameter_combination/hog_combination_rf.csv"
# Inicializar el archivo CSV
with open(nombre_archivo_csv, 'w', newline='') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['Orientations', 'Pixels per Cell', 'Accuracy', 'Recall', 'Specificity', 'Precision', 'F1-Score', 'AUC'])

@use_named_args(espacio_hog)
def funcion_objetivo(orientations, pixels_per_cell):
    hog_params = {'orientations': orientations, 'pixels_per_cell': (pixels_per_cell, pixels_per_cell)}
    datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), hog_params)
    datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), hog_params)
    modelo = RandomForestClassifier(random_state=42, n_jobs=-1)
    modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
    etiquetas_predichas = modelo.predict(datos_prueba)
    accuracy = accuracy_score(etiquetas_prueba, etiquetas_predichas)
    f1 = f1_score(etiquetas_prueba, etiquetas_predichas)
    precision = precision_score(etiquetas_prueba, etiquetas_predichas)
    recall = recall_score(etiquetas_prueba, etiquetas_predichas)
    cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    auc = roc_auc_score(etiquetas_prueba, modelo.predict_proba(datos_prueba)[:, 1])
    
    # Registrar en CSV
    with open(nombre_archivo_csv, 'a', newline='') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([orientations, pixels_per_cell, accuracy, recall, specificity, precision, f1, auc])
    
    return -f1

# Ejecutar la optimización
resultado = gp_minimize(funcion_objetivo, espacio_hog, n_calls=20, random_state=42, n_jobs=-1, verbose=True)

# Imprimir el mejor resultado
print(f"Mejor F1-Score: {-resultado.fun}")
print(f"Mejores hiperparámetros: Orientations={resultado.x[0]}, Pixels_per_cell=({resultado.x[1]}, {resultado.x[1]})")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 155.3929
Function value obtained: -0.8161
Current minimum: -0.8161
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 65.8667
Function value obtained: -0.8058
Current minimum: -0.8161
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 161.3111
Function value obtained: -0.8059
Current minimum: -0.8161
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 125.2108
Function value obtained: -0.8109
Current minimum: -0.8161
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 60.0657
Function value obtained: -0.8203
Current minimum: -0.8203
Iteration No: 6 s